# Package Import

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
from matplotlib import pyplot

from sklearn.feature_selection import SelectFromModel

# Dataset Astronomical Dataset

In [ ]:
# train = pd.read_csv('./data/train.csv', index_col=0)
# test = pd.read_csv('./data/test.csv', index_col=0)
# sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

# # Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
# column_number = {}
# for i, column in enumerate(sample_submission.columns):
#     column_number[column] = i
    
# def to_number(x, dic):
#     return dic[x]

# train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# # 모델에 적용할 데이터 셋 준비 
# x = train.drop(columns=['type', 'type_num'], axis=1)
# y = train['type_num']

# x = x.drop(columns=['fiberID'], axis=1)
# test_x = test.drop(columns=['fiberID'],axis=1)

# x_name=x.columns

# x=np.array(x)
# test_x = np.array(test_x)

# Dataset Iris (for test)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

iris_data =pd.read_csv("./iris2.csv",encoding='utf-8')

y=iris_data.loc[:,'Name']#.values
x=iris_data.iloc[:,:-1]#.values

x=x.values
y=y.values

# Date Read

In [ ]:
# x = np.load('./data/x.npy')
# y = np.load('./data/y.npy')
# pred = np.load('./data/pred.npy')

# LabelEncoder 

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(y)
y=enc.transform(y)

# to_categorical

In [ ]:
from keras.utils import to_categorical
#y=to_categorical(y)

# Train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8)

# StandardScaler, RobustScaler

In [ ]:
# from sklearn.preprocessing import RobustScaler, StandardScaler

# scaler1 = RobustScaler()
# scaler1.fit(x_train)
# x_train = scaler1.transform(x_train)
# x_test = scaler1.transform(x_test)

# scaler2 = RobustScaler()
# scaler2.fit(test_x)
# test_x = scaler2.transform(test_x)

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# scale=LabelEncoder()
# scale.fit(y)
# y=scale.transform(y)

# Model1 - RandomForest(RandomSearchCV)

In [ ]:
kfold_cv = KFold(n_splits=5, shuffle=True)
parameters =  {"n_estimators": [100,300,500], "max_depth":[4,8]}

clf1=RandomForestClassifier()
n_iter_search = 20

clf1=RandomizedSearchCV( clf1, param_distributions=parameters, cv=kfold_cv, \
                        n_iter=n_iter_search,n_jobs=4)
clf1.fit(x_train,y_train)

y_pred=clf1.predict(x_test)

print("acc", accuracy_score(y_test, y_pred))
print(clf1.best_params_)

In [ ]:
model1 =RandomForestClassifier(**clf1.best_params_)
#model1 =RandomForestClassifier()

model1.fit(x_train, y_train)

res=model1.score(x_test,y_test) 
print(res)

#y_pred = model1.predict_proba(test_x)

In [ ]:
try:
    plt.bar(range((len(model1.feature_importances_))), model1.feature_importances_)

except:
    mid=len(x_name)//2

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[:mid], model1.feature_importances_[:mid])
    plt.show()

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[mid:], model1.feature_importances_[mid:])

plt.show()

# Model2 - XGBClassifier(RandomSearchCV)

In [ ]:
kfold_cv = KFold(n_splits=5, shuffle=True)
parameters =  {"n_estimators": [200,400,600,800], "max_depth":[7,9,11], \
                "lr":[0.01,0.04,0.07,0.10], "feature_fraction":[0.6,0.9], \
                 "num_iterations":[500,800]}

clf2= XGBClassifier()
n_iter_search = 20

clf2=RandomizedSearchCV( clf2, param_distributions=parameters, cv=kfold_cv, \
                        n_iter=n_iter_search)

clf2.fit(x_train,y_train)

y_pred=clf2.predict(x_test)

print("acc", accuracy_score(y_test, y_pred))
print(clf2.best_params_)

In [ ]:
model2 =XGBClassifier(**clf2.best_params_)
#model2 = XGBClassifier()

eval_set = [(x_test, y_test)]
model2.fit(x_train, y_train, eval_metric="mlogloss", eval_set=eval_set, verbose=True,early_stopping_rounds=50)

res=model2.score(x_test,y_test) 
print(res)

#y_pred = model2.predict_proba(test_x)

In [ ]:
try:
    plt.bar(range((len(model2.feature_importances_))), model2.feature_importances_)

except:
    mid=len(x_name)//2

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[:mid], model2.feature_importances_[:mid])
    plt.show()

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[mid:], model2.feature_importances_[mid:])

plt.show()

In [ ]:
from xgboost import plot_importance
plot_importance(model2)

In [ ]:
# thresholds = np.sort(model2.feature_importances_)
# for thresh in thresholds:
#     # select features using threshold
#     selection = SelectFromModel(model2, threshold=thresh, prefit=True)
#     select_x_train = selection.transform(x_train)
#     # train model
#     selection_model = XGBClassifier()
#     selection_model.fit(select_x_train, y_train)
#     # eval model
#     select_x_test = selection.transform(x_test)
#     y_predict = selection_model.predict(select_x_test)
#     predictions = [round(value) for value in y_predict]
#     accuracy = accuracy_score(y_test, predictions)
#     print("Threshold=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

# Model3 - LGBMClassifier(RandomSearchCV)

In [ ]:
kfold_cv = KFold(n_splits=5, shuffle=True)
parameters =  {"n_estimators": [200,400,600,800], "max_depth":[7,9,11], \
                "lr":[0.01,0.04,0.07,0.10], "feature_fraction":[0.6,0.9], \
                 "num_iterations":[500,800]}

clf3= LGBMClassifier()
n_iter_search = 20

clf3=RandomizedSearchCV( clf3, param_distributions=parameters, cv=kfold_cv, \
                        n_iter=n_iter_search,n_jobs=4)
clf3.fit(x_train,y_train)

y_pred=clf3.predict(x_test)

print("acc", accuracy_score(y_test, y_pred))
print(clf3.best_params_)

In [ ]:
model3 = LGBMClassifier(**clf3.best_params_)
#model3 = LGBMClassifier()

eval_set = [(x_test, y_test)]
model3.fit(x_train, y_train, eval_metric="multi_logloss", eval_set=eval_set, verbose=True,early_stopping_rounds=25)

res=model3.score(x_test,y_test) 
print(res)

#y_pred = model3.predict_proba(test_x)

In [ ]:
try:
    plt.bar(range((len(model3.feature_importances_))), model3.feature_importances_)

except:
    mid=len(x_name)//2

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[:mid], model3.feature_importances_[:mid])
    plt.show()

    plt.figure(figsize=(15, 6))
    plt.bar(x_name[mid:], model3.feature_importances_[mid:])

plt.show()

In [ ]:
from lightgbm import plot_importance
plot_importance(model3)